In [ ]:
#With handtracking, masi ada bug blm kelar

In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import mediapipe as mp


2024-02-14 23:39:11.453095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
input_dir = './data/'
output_dir = './data2/'

In [4]:
frame_rate = 1
def extract_frame(video_path, output_dir, frame_rate):
    video = cv2.VideoCapture(video_path)
    video_fps = int(video.get(cv2.CAP_PROP_FPS))
    success, frame = video.read()
    frame_count = 0

    while success:
        if frame_count % (video_fps // frame_rate) == 0:
            frame_path = os.path.join(output_dir, f'{frame_count}.jpg')
            cv2.imwrite(frame_path, frame)

        success, frame = video.read()
        frame_count += 1
    
    video.release()

for subfolder in os.listdir(input_dir):
    subfolder_path = os.path.join(input_dir, subfolder)

    # Make sure it's a directory
    if os.path.isdir(subfolder_path):
        # Loop through each video file in the subfolder
        for video_file in os.listdir(subfolder_path):
            video_path = os.path.join(subfolder_path, video_file)
            # Check if the file is a video
            if os.path.isfile(video_path) and video_path.endswith(('.mp4', '.MOV')):
                # Create a corresponding output folder for the frames

                output_folder_path = os.path.join(subfolder_path, video_file.split('.')[0] + '_frames')
                if not os.path.exists(output_folder_path):
                    os.makedirs(output_folder_path)
                # Extract and save frames
                extract_frame(video_path, output_folder_path, frame_rate)

In [ ]:
frame_rate = 1

def extract_frames(video_path, output_dir, frame_rate):
    video = cv2.VideoCapture(video_path)
    video_fps = int(video.get(cv2.CAP_PROP_FPS))
    success, frame = video.read()
    frame_count = 0

    while success:
        if frame_count % (video_fps // frame_rate) == 0:
            frame_path = os.path.join(output_dir, f'{frame_count}.jpg')
            cv2.imwrite(frame_path, frame)

        success, frame = video.read()
        frame_count += 1
    
    video.release()


def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Walk through the input dataset directory and process videos
for set_name in os.listdir(input_dir):
    set_path = os.path.join(input_dir, set_name)
    for class_name in os.listdir(set_path):
        class_path = os.path.join(set_path, class_name)
        if not os.path.isdir(class_path): 
            continue
        output_class_path = os.path.join(output_dir, set_name, class_name)
        ensure_dir(output_class_path)

        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            if not video_name.lower().endswith(('.mp4', '.MOV')):  # Check for video files
                continue
            output_video_path = os.path.join(output_class_path, video_name.split('.')[0])
            ensure_dir(output_video_path)

            extract_frames(video_path, output_video_path, frame_rate)

print("Frame extraction completed.")    

In [6]:
import pickle

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands( min_detection_confidence=0.5, min_tracking_confidence=0.5)

data = []
labels = []
for dir_ in os.listdir(output_dir):
    for img_path in os.listdir(os.path.join(output_dir, dir_)):
        data_aux = []
        img = cv2.imread(os.path.join(output_dir, dir_, img_path))
        im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(im_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y 
                    data_aux.append(x)
                    data_aux.append(y)
            data.append(data_aux)
            labels.append(dir_)


f = open("datasets2.pickle", "wb")
pickle.dump({"data": data, "label": labels}, f)    
f.close()                                 

I0000 00:00:1707720863.614101       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) Iris(TM) Plus Graphics OpenGL Engine
